In [1]:
import torch
import pandas as pd
import numpy as np
import os, sys
import time

import matplotlib.pyplot as plt
sys.path.append(os.path.realpath('./src/'))
from materialEncoder import MaterialEncoder


from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon, Ellipse
from smallestEllipse import *

%matplotlib qt


In [2]:
def check_xl():
    df = pd.read_excel('./data/newMaterialData_7D.xlsx')# solidworksMaterialDatabaseCost # aluminum
    return df

df = check_xl()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Material         50 non-null     object 
 1   H (Mohs)/H0      50 non-null     float64
 2   E(GPa)/E0        50 non-null     float64
 3   UTS (MPa)        50 non-null     float64
 4   Rho (g/cc)/Rho0  50 non-null     float64
 5   TM (degC)/T0     50 non-null     float64
 6   CTE (1/K)/CTE0   50 non-null     float64
 7   TC (W/m-K)/TC0   50 non-null     float64
dtypes: float64(7), object(1)
memory usage: 3.3+ KB


In [3]:
x = df['Rho (g/cc)/Rho0'].values
y = df['E(GPa)/E0'].values
txt = df['Material'].values
arr = np.array([0]*25 + [1]*25)
clrs = ['red' if val == 0 else 'blue' for val in arr]


print(txt)
fig, ax = plt.subplots()
ax.scatter(x, y, c=clrs)

for i, j in enumerate(txt):
    ax.annotate(j, (x[i], y[i]))
    
from smallestEllipse import *

for i in [0,1]:
    pts = np.array([x[25*i:25*(i+1)], y[25*i:25*(i+1)]]).T
    enclosing_ellipse = welzl(np.array(pts, dtype=float))
    # plot resulting ellipse
    center,a,b,t = enclosing_ellipse
    elli = plot_ellipse(enclosing_ellipse, str='k')
    ellipse = Ellipse(xy=center, width=2*a, height=2*b, angle=np.degrees(t), edgecolor='k', fc='r' if i == 0 else 'b', alpha=0.3, lw=2)
    ax.add_patch(ellipse)

ax.set_xlabel('Density')
ax.set_ylabel('Youngs Modulus')

['Indium' 'Lead' 'Tin' 'Titanium' 'Tungsten' 'Molybdenum' 'Zirconium'
 'Tantalum' 'Gold' 'Silver' 'Lanthanum' 'Cadmium' 'Zinc' 'Aluminum'
 'Antimony' 'Platinum' 'Palladium' 'Nickel' 'Vanadium' 'Iron' 'Copper'
 'Chromium' 'Rhodium' 'Magnesium' 'Beryllium' 'Alumina' 'WC'
 'Barium Titanate' 'AlN' 'Zirconia' 'MgO' 'Silicon Nitride' 'BeO'
 'Boron Carbide' 'Titanium Diboride' 'Boron Nitride' 'Mullite'
 'Tantalum Carbide' 'Cordierite' 'a-SiC' 'Fused Silica' 'Titanium Nitride'
 'Forsterite' 'TiC' 'Calcium Fluoride' 'Steatite' 'Soda-lime glass'
 'Borosilicate glass' 'Brick' 'Concrete']


Text(0, 0.5, 'Youngs Modulus')

In [4]:
combined_array = np.column_stack((x, y))

# Convert the NumPy array to a PyTorch tensor
d = torch.tensor(combined_array, dtype=torch.float32)

min_vals = d.min(dim=0, keepdim=True)[0]  # Minimum values for each column
max_vals = d.max(dim=0, keepdim=True)[0]  # Maximum values for each column

# Apply normalization
data = (d - min_vals) / (max_vals - min_vals)

print(data)

tensor([[0.2822, 0.0000],
        [0.4876, 0.0022],
        [0.2817, 0.0515],
        [0.1401, 0.1759],
        [0.8909, 0.6595],
        [0.4303, 0.5403],
        [0.2431, 0.1393],
        [0.7565, 0.2951],
        [0.8919, 0.1098],
        [0.4440, 0.1078],
        [0.2248, 0.0407],
        [0.3501, 0.0724],
        [0.2720, 0.1427],
        [0.0488, 0.0959],
        [0.2476, 0.1108],
        [1.0000, 0.2644],
        [0.5218, 0.1844],
        [0.3637, 0.3189],
        [0.2218, 0.1963],
        [0.3113, 0.3376],
        [0.3664, 0.1997],
        [0.2766, 0.4534],
        [0.5434, 0.4466],
        [0.0000, 0.0550],
        [0.0056, 0.4671],
        [0.1061, 0.6475],
        [0.6982, 1.0000],
        [0.2172, 0.0925],
        [0.0772, 0.5403],
        [0.2172, 0.3189],
        [0.0945, 0.4892],
        [0.0726, 0.3734],
        [0.0645, 0.5658],
        [0.0346, 0.6799],
        [0.1396, 0.6935],
        [0.0184, 0.0805],
        [0.0539, 0.2219],
        [0.6474, 0.6169],
        [0.0

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Define the Encoder
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(2, 50)
        self.fc21 = nn.Linear(50, 2)
        self.fc22 = nn.Linear(50, 2)

    def forward(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

# Define the Decoder
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc3 = nn.Linear(2, 50)
        self.fc4 = nn.Linear(50, 2)

    def forward(self, z):
        h3 = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

# Define the VAE
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        mu, logvar = self.encoder(x.view(-1, 2))
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

# Define the loss function
def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 2), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    beta = 5e-5
    return BCE + beta*KLD

# Training setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Dummy dataset

# Training loop
model.train()
for epoch in range(1000):  # Number of epochs
    optimizer.zero_grad()
    recon_batch, mu, logvar = model(data)
    loss = loss_function(recon_batch, data, mu, logvar)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Plotting the latent space
model.eval()

with torch.no_grad():
    _, mu, _ = model(data)
    plt, ax = plt.subplots()
    ax.scatter(mu[:, 0].cpu(), mu[:, 1].cpu(), c=clrs, marker='o')
    ax.set_title('Latent Space')
    ax.set_xlabel('Latent Dimension 1')
    ax.set_ylabel('Latent Dimension 2')
    for i,j in enumerate(txt):
        ax.annotate(j, (mu[i, 0].cpu().numpy(), mu[i, 1].cpu().numpy()))


for i in [0,1]:
    pts = np.array([mu[25*i:25*(i+1),0].cpu().numpy(), mu[25*i:25*(i+1),1].cpu().numpy()]).T
    enclosing_ellipse = welzl(np.array(pts, dtype=float))
    # plot resulting ellipse
    center,a,b,t = enclosing_ellipse
    elli = plot_ellipse(enclosing_ellipse, str='k')
    ellipse = Ellipse(xy=center, width=2*a, height=2*b, angle=np.degrees(t), edgecolor='k', fc='r' if i == 0 else 'b', alpha=0.3, lw=2)
    ax.add_patch(ellipse)



Epoch 1, Loss: 73.01908874511719
Epoch 2, Loss: 70.69432067871094
Epoch 3, Loss: 71.22345733642578
Epoch 4, Loss: 69.61856079101562
Epoch 5, Loss: 69.81043243408203
Epoch 6, Loss: 70.60877227783203
Epoch 7, Loss: 69.3958969116211
Epoch 8, Loss: 68.55654907226562
Epoch 9, Loss: 68.64385986328125
Epoch 10, Loss: 69.24551391601562
Epoch 11, Loss: 69.59916687011719
Epoch 12, Loss: 67.3238754272461
Epoch 13, Loss: 66.5539321899414
Epoch 14, Loss: 67.81016540527344
Epoch 15, Loss: 67.02982330322266
Epoch 16, Loss: 66.54633331298828
Epoch 17, Loss: 67.61050415039062
Epoch 18, Loss: 66.04324340820312
Epoch 19, Loss: 66.36332702636719
Epoch 20, Loss: 64.8904037475586
Epoch 21, Loss: 65.39141845703125
Epoch 22, Loss: 64.7639389038086
Epoch 23, Loss: 63.08685302734375
Epoch 24, Loss: 64.98759460449219
Epoch 25, Loss: 65.69403076171875
Epoch 26, Loss: 63.41567611694336
Epoch 27, Loss: 63.069122314453125
Epoch 28, Loss: 63.50511169433594
Epoch 29, Loss: 63.666988372802734
Epoch 30, Loss: 62.8144950

In [19]:
plt.close('all')

AttributeError: 'Figure' object has no attribute 'close'

In [ ]:
txt